# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f074446b9a0>
├── 'a' --> tensor([[-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290]])
└── 'x' --> <FastTreeValue 0x7f074446b2b0>
    └── 'c' --> tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                        [-0.4691,  0.1119,  0.5934,  0.9626],
                        [-0.7331, -0.0108,  0.5947,  1.3659]])

In [4]:
t.a

tensor([[-0.4648,  0.1993,  1.8908],
        [-0.4757,  0.4677,  0.6290]])

In [5]:
%timeit t.a

78.8 ns ± 0.986 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f074446b9a0>
├── 'a' --> tensor([[-0.2698,  0.5617, -1.0780],
│                   [-1.4054,  0.3285,  0.1758]])
└── 'x' --> <FastTreeValue 0x7f074446b2b0>
    └── 'c' --> tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                        [-0.4691,  0.1119,  0.5934,  0.9626],
                        [-0.7331, -0.0108,  0.5947,  1.3659]])

In [7]:
%timeit t.a = new_value

72.3 ns ± 1.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4648,  0.1993,  1.8908],
               [-0.4757,  0.4677,  0.6290]]),
    x: Batch(
           c: tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659]]),
       ),
)

In [10]:
b.a

tensor([[-0.4648,  0.1993,  1.8908],
        [-0.4757,  0.4677,  0.6290]])

In [11]:
%timeit b.a

67.6 ns ± 2.53 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3579,  0.9733, -0.4450],
               [-1.3282,  0.5019,  0.6190]]),
    x: Batch(
           c: tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659]]),
       ),
)

In [13]:
%timeit b.a = new_value

576 ns ± 9.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

280 µs ± 9.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 7.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0693355bb0>
├── 'a' --> tensor([[[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]],
│           
│                   [[-0.4648,  0.1993,  1.8908],
│                    [-0.4757,  0.4677,  0.6290]]])
└── 'x' --> <FastTreeValue 0x7f0746548a60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.2 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f068c6fd0a0>
├── 'a' --> tensor([[-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290],
│                   [-0.4648,  0.1993,  1.8908],
│                   [-0.4757,  0.4677,  0.6290]])
└── 'x' --> <FastTreeValue 0x7f0693361610>
    └── 'c' --> tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                        [-0.4691,  0.1119,  0.5934,  0.9626],
                 

In [23]:
%timeit t_cat(trees)

41.7 µs ± 786 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.9 µs ± 508 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2808, -0.0065,  1.2162, -0.9202],
                       [-0.4691,  0.1119,  0.5934,  0.9626],
                       [-0.7331, -0.0108,  0.5947,  1.3659]],
              
                      [[ 0.2808, -0.0065,  1.2162, -0.9202],
                       [-0.4691,  0.1119,  0.5934,  0.9626],
                       [-0.7331, -0.0108,  0.5947,  1.3659]],
              
                      [[ 0.2808, -0.0065,  1.2162, -0.9202],
                       [-0.4691,  0.1119,  0.5934,  0.9626],
                       [-0.7331, -0.0108,  0.5947,  1.3659]],
              
                      [[ 0.2808, -0.0065,  1.2162, -0.9202],
                       [-0.4691,  0.1119,  0.5934,  0.9626],
                       [-0.7331, -0.0108,  0.5947,  1.3659]],
              
                      [[ 0.2808, -0.0065,  1.2162, -0.9202],
                       [-0.4691,  0.1119,  0.5934,  0.9626],
                       [-0.7331, -0.0108,  0.5947,  1.3659]],

In [26]:
%timeit Batch.stack(batches)

103 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659],
                      [ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659],
                      [ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659],
                      [ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659],
                      [ 0.2808, -0.0065,  1.2162, -0.9202],
                      [-0.4691,  0.1119,  0.5934,  0.9626],
                      [-0.7331, -0.0108,  0.5947,  1.3659],
                      [ 0.2808, -0.0065,  1.2162, -0.9202],
                   

In [28]:
%timeit Batch.cat(batches)

182 µs ± 7.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

518 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
